<a href="https://colab.research.google.com/github/ngockhanh5110/nlp-vietnamese-text-summarization/blob/main/notebooks/inferencing_vietnews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Overview: The code below is for the purpose of comparing the quality of the algorithms Lsa, KL, LexRank, TextRank, SumBasic, Luhn with the model I trained**

# **Connect to drive**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **Install lib**

In [ ]:
!pip install vncorenlp datasets==1.0.2
!pip install transformers
!pip install dill==0.3.5.1
!pip install torch



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 29.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 44.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 25.2 MB/s eta 0:00:00
  Created wheel for vncorenlp: filename=vncorenlp-1.0.3-py3-none-any.whl size=2645933 sha256=21bf8bfe6b5b71e5c6de575b97d20588176eda7db5e7d098a30ef5b26ce786dc
  Stored in directory: /root/.cache/pip/wheels/5d/d9/b3/41f6c6b1ab758561fd4aab55dc0480b9d7a131c6aaa573a3fa
Successfully built vncorenlp
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 43.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 48.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install git-python==1.0.3
# !pip install sacrebleu==1.4.12
!pip install rouge_score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=fe291713a9372e30bd5bbbecafbfd6618d813773f10763d06dfa5b3683375825
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [ ]:
import os
new_path = "/content/drive/MyDrive/LLM_TEXT/"
os.chdir(new_path)
current_path = os.getcwd()
print("current path:", current_path)

current path: /content/drive/MyDrive/LLM_TEXT


In [ ]:
import glob
import pandas as pd
import concurrent.futures
from datasets import *
import datasets
import transformers
import nltk
nltk.download('punkt')

from transformers import RobertaTokenizerFast,AutoTokenizer
from seq2seq_trainer import Seq2SeqTrainer
from transformers import TrainingArguments
from dataclasses import dataclass, field
from typing import Optional

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
!pip install sumy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.3/97.3 kB 865.1 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 18.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for breadability: filename=breadability-0.1.20-py2.py3-none-any.whl size=21691 sha256=42eb4ea4b8ddc6322f66dbf4ff9ac8b072680c80abce1993777ecf604f2324df
  Stored in directory: /root/.cache/pip/wheels/64/22/90/b84fcc30e16598db20a0d41340616dbf9b1e82bbcc627b0b33
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=8fb4c18f040ce52bc1c2cebe4271ed4f2be169c8c2d9da2831545bfcc939c3b2
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
  Created wheel for pycountry: filename=pycountry-22.3.5-py2.py3-none-any.whl size=106

In [ ]:
!pip install dateparser
!pip install datefinder

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.8/293.8 kB 4.2 MB/s eta 0:00:00


In [ ]:
listFile = ["07.txt", "08.txt", "09.txt", "10.txt", "11.txt", "12.txt", "13.txt"]

# **Processing data**

In [ ]:
size_data_test = 50
summary_task_original_new = []

In [ ]:
# Đọc danh sách các file từ đường dẫn được cung cấp
def listPaths(path):
  pathfiles = list()
  cnt = 0
  for pathfile in glob.glob(path):
    cnt += 1
    file_name = os.path.basename(pathfile)
    for file_ in  listFile:
      if file_name == file_:
        pathfiles.append(pathfile)
        break
    if cnt == size_data_test:
       break
  return pathfiles

test_paths = listPaths('/content/drive/MyDrive/LLM_TEXT/dataset/BBCHnew/*.txt')
# for file in test_paths:
#   print(file)

In [ ]:
# Đọc nội dung file
def read_content(path_file):
  """
  Input: Path of txt file
  Output
  """
  # print("\n\n path_file:", path_file)
  summary_content = ""
  summary_task = ""
  original = ""
  summary_taskDict = []
  with open(path_file, "r", encoding="utf-8") as file:
      lines  = file.readlines()
      cnt = 0
      # print("\n\n path_file:",path_file)
      # print("\n\n lines:",lines)
      for line in lines:
          line = line.strip()

          if "NOI_DUNG_TOM_TAT" in line:
              cnt = 1
              line = line.replace("NOI_DUNG_TOM_TAT", "")
              line = line.replace(":", "")
          elif "NHIEM_VU" in line:#line.startswith("NHIEM_VU"):
              cnt = 2
              line = line.replace("NHIEM_VU", "")
              line = line.replace(":", "")
          # new code
          elif "NGUOI_LAM" in line: #line.startswith("NGUOI_LAM"):
              summary_taskDict.append({ "people": "", "content": "", "time": ""})
              cnt = 20
              line = line.replace("NGUOI_LAM", "")
              line = line.replace(":", "")
          elif "NOI_DUNG_CV" in line: #line.startswith("NOI_DUNG_CV"):
              cnt = 21
              line = line.replace("NOI_DUNG_CV", "")
              line = line.replace(":", "")
          elif "THOI_GIAN" in line: #line.startswith("THOI_GIAN"):
              cnt = 22
              line = line.replace("THOI_GIAN", "")
              line = line.replace(":", "")
          #end
          elif "NOI_DUNG_GOC" in line: #line.startswith("NOI_DUNG_GOC"):
              cnt = 3
              line = line.replace("NOI_DUNG_GOC", "")
              line = line.replace(":", "")

          line += " "
          # line  = line.replace("..", ". ")
          if line:
            if cnt == 1:
              summary_content += line
            #elif cnt == 2:
            #  summary_task += line
            elif cnt == 20:
              summary_taskDict[-1]["people"] += line
            elif cnt == 21:
              summary_taskDict[-1]["content"] += line
            elif cnt == 22:
              summary_taskDict[-1]["time"] += line

            elif cnt == 3:
              original += line

  summary_task_original_new.append({'file' : path_file,
          'summary_task': summary_taskDict}) # chứa các nội dung people: tên người làm, time: thời gian, và content: nội dung nhiệm vụ

  return {'file' : path_file,
          # 'summary_task': summary_taskDict,
          'summary_content': summary_content, # chứa nội dung tóm tắt cuộc họp
          'original': original} # văn bản cuộc họp gốc ( văn bản cần tóm tắt (trích xuất))

In [ ]:
# Đọc các key word
def get_stop_words_vn(path):
    ret = []
    # path = "/content/drive/MyDrive/LLM_TEXT/vietnamese-stopwords-dash.txt"
    f = open(path, "r")
    lines = f.readlines()
    for line  in lines:
       line = line.replace("_", " ")
       line = " " + line.replace("\n", "") + " "
       ret.append(line)
    return ret

stop_words = get_stop_words_vn("/content/drive/MyDrive/LLM_TEXT/vietnamese-stopwords-dash.txt") # chứa các keyword cho việc xóa các từ dừng trong tiếng việt
task_words = get_stop_words_vn("/content/drive/MyDrive/LLM_TEXT/vietnamese-task-dash.txt") # chứa các keyword dành cho trích xuất nhiệm vụ

# **Get data**

In [ ]:
def get_dataframe(pathfiles):
  with concurrent.futures.ProcessPoolExecutor() as executor:
    data = executor.map(read_content, pathfiles)

  # Make blank dataframe
  data_df = list()
  for d in data:
    data_df.append(d)
  data_df = pd.DataFrame(data_df)
  data_df.dropna(inplace = True)
  data_df = data_df.sample(frac=1).reset_index(drop=True)

  return data_df

test_df = get_dataframe(test_paths)
summary_task_original_new = []
for path_file in test_df["file"]:
  read_content(path_file)

## **Summary task**

In [ ]:
import nltk
from nltk import sent_tokenize
# nltk.download('punkt')
import torch
from transformers import AutoTokenizer, AutoModel
from sklearn.metrics.pairwise import cosine_similarity

# đọc model PhoBERT cho việc trích xuất nhiệm vụ
tokenizer_nvu = AutoTokenizer.from_pretrained('vinai/phobert-base')
model_nvu = AutoModel.from_pretrained('vinai/phobert-base')

# chức năng chia nhỏ đoạn văn thành các đoạn nhỏ sử dụng text segmentation in NLP
def content_sentences( body: str, min_length=5, max_length=200):
        sents = [s.strip() for s in sent_tokenize(body)]
        return [c for c in sents if max_length > len(c.split(" ")) > min_length]

# Sử dụng PhoBERT để convert text thành các dạng vector
def encode_sentences(sentences):
    encoded_inputs = tokenizer_nvu(sentences, padding=True, truncation=True, return_tensors='pt')
    with torch.no_grad():
        model_output = model_nvu(**encoded_inputs)
    return model_output.pooler_output

# Đánh giá sự tương đồng của 2 đoạn (về nội dung, ngữ nghĩa) => trả về giá trị 0 - 1.0
def calculate_cosine_similarity(sentence_infer_embedding, sentence2):
    sentence2_embedding = encode_sentences(sentence2)
    similarity_score = cosine_similarity(sentence_infer_embedding, sentence2_embedding)
    return similarity_score[0][0]



# Ví dụ:
sentence1 = "Quỳnh nhận nhiệm vụ sẽ tuyên truyền giáo dục An toàn giao thông cho học sinh vào các giờ chào cờ đầu tuần."
sentence2 = "Trường THCS kim Liên diễn ra cuộc họp của ban trung tâm bàn nội dung công việc và nhiệm vụ thực hiện của tháng 5.."

sentence_infer_embedding = encode_sentences(sentence1)
similarity_percentage = calculate_cosine_similarity(sentence_infer_embedding, sentence2)
print(f"Độ giống nhau: {similarity_percentage:.2f}%")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Độ giống nhau: 0.47%


In [ ]:
inferrence_Task_embedding = [] # Tạo list data để làm dữ liệu tham chiếu cho việc trích xuất nhiệm vụ, chứa các mã encoder cho việc trích xuất nhiệm vụ

for listT in summary_task_original_new:
    for content in listT['summary_task']:
      content_ = content['content']
      sentences = content_sentences(content_, 1, 400)
      for sentence in sentences:
         print("sentence ", sentence )
         encode_sentence_ = encode_sentences(sentence)
         inferrence_Task_embedding.append(encode_sentence_)
         break

print("length of inferrence_Task_embedding: ", len(inferrence_Task_embedding))

sentence  Lan chủ tịch công đoàn là chủ tọa  điều hành cuộc họp và phân công nhiệm vụ cụ thể  cho các thành viên.
sentence  Hương đảm nhận  kiểm tra, giám sát việc thực hiện điều lệ, kế hoạch, nghị quyết của CĐCS.
sentence  Mạnh đảm nhận xây dựng kế hoạch cho ngày kỉ niệm 76 Quốc khánh nước cộng hoà xã hội chủ nghĩa Việt Nam
sentence  Kiên đảm nhận kết hợp với các đồng chí Lan, và Hương thực hiện tốt nhiệm vụ được phân công
sentence  Ông Tùng là tổ trưởng chuyên môn  điều hành cuộc họp và  phân công nhiệm vụ cho các thành viên trong tổ nghiên cứu, thảo luận và đánh giá  theo tiêu chí  các văn bản hướng dẫn sau Thông tư số 31/2021/TT-BGDĐT ngày 23/08/2021 của Bộ Giáo dục và Đào tạo Quy định việc lựa chọn sách giáo khoa trong cơ sở giáo dục phổ thông.
sentence  Huệ  đảm nhận tìm hiểu về cấu trúc các bài trong một chương và mối liên hệ giữa các bài học.
sentence  Hạnh đảm nhận tìm hiểu nội dung hướng nghiệp.
sentence  đảm nhận nội dung giáo dục An toàn giao thông
sentence  Linh sẽ đảm nhậ

In [ ]:
# Tham số threshould: dùng để làm ngưỡng đánh giá các câu là hợp lệ (trong quá trình làm thực tế tự nhận thấy threshold = 0.9 là đạt hiệu quả tốt về chất lượng các câu được cho là đúng)
def compare_task(sentence, threshould_ = 0.9):
  pre = -1
  for task_infer in inferrence_Task_embedding:
    ret = calculate_cosine_similarity(task_infer, sentence)
    if ret > pre:
      pre = ret
    if ret > threshould_ :
      return ret
  return pre

### **Case 1: Using vnvorenlp** (get name)



In [ ]:
!pip install vncorenlp

In [ ]:
import os
from vncorenlp import VnCoreNLP
vncorenlp_folder = '/content/drive/MyDrive/LLM_TEXT/VnCoreNLP-master/'
vncorenlp_file = os.path.join(vncorenlp_folder, 'VnCoreNLP-1.1.1.jar')

vnlp = VnCoreNLP(vncorenlp_file, annotators="wseg,pos,ner")

In [ ]:
text = "Hôm nay ngày 21 tháng 4 năm 2023 tại trường THCS kim Liên diễn ra cuộcHôm nay, Mai Anh và Hải Đăng đã đến thăm chị Lan tại Hà Nội. họp của ban trung tâm bàn nội "
def getName(str):
  annotated_text = vnlp.annotate(str)
  person_entities = []
  for sentence in annotated_text['sentences']:
      for entity in sentence:
          if entity["nerLabel"] == 'B-PER':
              person_entities.append(entity["form"])
  return person_entities

# Kiểm tra hàm getName()
print(getName(text))

['Mai_Anh', 'Hải_Đăng', 'Lan']


### **Case 2: Pre-train model (PhoBERT)** (get name)

In [ ]:
import datasets
from transformers import RobertaTokenizer, EncoderDecoderModel, AutoTokenizer
from sklearn.model_selection import train_test_split

#
tokenizerName = AutoTokenizer.from_pretrained("vinai/phobert-base", use_fast=False)

modelName = EncoderDecoderModel.from_pretrained('/content/drive/MyDrive/LLM_TEXT/trainning_Name/checkpoint-8000')
modelName.to("cpu")
batch_size = 32  # change to 64 for full evaluation



Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
The following encoder weights were not tied to the decoder ['roberta/pooler']
The following encoder weights were not tied to the decoder ['roberta/pooler']
The following encoder weights were not tied to the decoder ['roberta/pooler']


In [ ]:
# map data correctly
def getNameCase2(strInput):
    ret = []
    inputs = tokenizerName(strInput, padding="max_length", truncation=True, max_length=256, return_tensors="pt")
    input_ids = inputs.input_ids.to("cpu") #cuda
    attention_mask = inputs.attention_mask.to("cpu")
    outputs = modelName.generate(input_ids, attention_mask=attention_mask)
    output_str = tokenizerName.batch_decode(outputs, skip_special_tokens=True)

    # Lọc lại kết quả tên
    for name in output_str:
      if name in strInput:
        ret.append(name)
    return ret

## **Extract time**

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForMaskedLM
import dateparser
import nltk
nltk.download('averaged_perceptron_tagger')
# Load AutoTokenizer and AutoModel
tokenizer_time = AutoTokenizer.from_pretrained("vinai/phobert-base")
model_time = AutoModelForMaskedLM.from_pretrained("vinai/phobert-base")

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Some weights of the model checkpoint at vinai/phobert-base were not used when initializing RobertaForMaskedLM: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
import nltk
import re
import datefinder

import re

def extract_month(text):
  date_pattern = r"tháng\s(\d{1,2})"
  matches = re.findall(date_pattern, text)
  try:
    mon = int(matches[0])
    return mon
  except:
    pass
  return None

#Sử dụng datefinder để trích xuất thời gian
def extract_time_datefinder(str_):
  # print("str_: ", str_)
  date_str = {}
  matches = []
  matches = datefinder.find_dates(str_)
  dt = list(matches)
  # print("dt", dt)
  for time in dt:
    day = time.day
    month = time.month
    year = time.year
    month_ = extract_month(str_)
    if month_ is not None:
       month =  month_
    data_ = {"day": str(day), "month": str(month), "year": str(year)}
    # date_str.append(data_)
    return data_
  return None

# sử dụng nlp để trích xuất thời gian
def extract_time_nlp(text):
  tokens = nltk.word_tokenize(text)
  time_expressions = [token for token in tokens if nltk.pos_tag([token])[0][1] == "TIME"]
  if not time_expressions:
    return None
  return time_expressions[0]

#Sử dụng PhoBERT để trích xuất thời gian
def extract_time_phobert(text):
  input_ids = tokenizer_time.encode(text, add_special_tokens=True)
  input_ids = torch.tensor(input_ids).unsqueeze(0)
  with torch.no_grad():
      outputs = model_time(input_ids)
  parsed_date = dateparser.parse(text, languages=["vi"])

#Hàm trích xuất thời gian
def extract_time(text):
  time = extract_time_nlp(text)
  if time is not None:
    return time

  time = extract_time_phobert(text)
  if time is not None:
    return time

  time = extract_time_datefinder(text)
  if time is not None:
    return time

text = " Deadline ngày 22 tháng 5, 2023" # Deadline 22 tháng 5, 2023.   ', ' Deadline 22 tháng 5, 2023.  ', ' Deadline 17 tháng 5, 2023.

# Extract the time from the text.
time1 = extract_time_datefinder(text)

# Print the time.
print("time1: ", time1)
# print("time1: ", time1["day"])

time1:  {'day': '22', 'month': '5', 'year': '2023'}


### **Summary Task**

In [ ]:
import nltk
from nltk import sent_tokenize
# nltk.download('punkt')
import re
THRESHOLD = 0.9
def contains_digits(input_str):
    digit_pattern = r'\d'
    match = re.search(digit_pattern, input_str)
    return match is not None

def process_content_sentences( body: str, min_length=5, max_length=50):
        # for word in reversed(stop_words):
        #   body.replace(word, " ")
        sents = [s.strip() for s in sent_tokenize(body)]
        return [c for c in sents if max_length > len(c.split(" ")) > min_length]

#Hàm trích xuất nhiệm vụ
def detectTask(str, case1 = True):
  #Phân đoạn văn bản đầu vào
  sentences = process_content_sentences(str, 1, 400)
  ret = []
  for sentence in sentences:
    sentencelower = sentence.lower()
    time = ""

    # Trích xuất nhiệm vụ
    preci_ = compare_task(sentence, THRESHOLD)
    # for wTask in task_words:
    #   if wTask.strip() in sentencelower:
    if preci_ >= THRESHOLD:
      people = []

      # Trích xuất tên người
      if case1:
        people = getName(sentence) # Sử dụng VncoreNLP
      else:
        people = getNameCase2(sentence) # Sử dụng PhoBERT

      ret.append({"people": people, "content": sentencelower, "time": {'day': -1 , 'month': -1, 'year': -1} })

    # Trích xuất thời gian
    time_ =  extract_time(sentence)
    if time_ is not None:
      daystr = ""
      try:
        if len(ret) > 0:
          daystr = time_["day"]
          if daystr.strip() in sentencelower:
            ret[-1]["time"]["day"] = time_["day"]
            ret[-1]["time"]["month"] = time_["month"]
            ret[-1]["time"]["year"] = time_["year"]
      except:
        print(" errr")
        pass

  return ret

def detectTaskeyword(str, case1 = True):
  sentences = process_content_sentences(str, 1, 400)
  ret = []
  for sentence in sentences:
    sentencelower = sentence.lower()
    time = ""

    # Trích xuất nhiệm vụ sử dụng keyword
    for wTask in task_words:
      if wTask.strip() in sentencelower:
        people = []
        if case1:
          people = getName(sentence)
        else:
          people = getNameCase2(sentence)

        ret.append({"people": people, "content": sentencelower, "time": {'day': -1 , 'month': -1, 'year': -1} })
        break
    time_ =  extract_time(sentence)
    if time_ is not None:
      daystr = ""
      try:
        if len(ret) > 0:
          daystr = time_["day"]
          if daystr.strip() in sentencelower:
            ret[-1]["time"]["day"] = time_["day"]
            ret[-1]["time"]["month"] = time_["month"]
            ret[-1]["time"]["year"] = time_["year"]
      except:
        print(" errr")
        pass

  return ret

# **summary content**

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning, message="Using `max_length`'s default")


In [ ]:
import datasets
from transformers import RobertaTokenizer, EncoderDecoderModel, AutoTokenizer
from sklearn.model_selection import train_test_split

# load model
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base", use_fast=False)

model = EncoderDecoderModel.from_pretrained('/content/drive/MyDrive/LLM_TEXT/trainning_summary_content_bbch/checkpoint-8000')
model.to("cpu")
batch_size = 32  # change to 64 for full evaluation

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
The following encoder weights were not tied to the decoder ['roberta/pooler']
The following encoder weights were not tied to the decoder ['roberta/pooler']
The following encoder weights were not tied to the decoder ['roberta/pooler']


In [ ]:
# tóm tắt nội dung sử dụng PhoBERT
def generate_summary(strOriginal):
    # Tokenizer will automatically set [BOS] <text> [EOS]
    inputs = tokenizer(strOriginal, padding="max_length", truncation=True, max_length=256, return_tensors="pt")
    input_ids = inputs.input_ids.to("cpu")
    attention_mask = inputs.attention_mask.to("cpu")
    outputs = model.generate(input_ids, attention_mask=attention_mask)
    output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    return output_str

# **Demo**

In [ ]:
def inference(inputText):
  print("Văn bản gốc: ", inputText)
  cntTask = 0
  summaryContent = generate_summary(inputText)
  print("\nNội dung cuộc họp được tóm tắt: ", summaryContent)

  #Trích xuất nhiệm vụ Sử dụng cosine_similarity
  print("\nTrích xuất nhiệm vụ sử dụng cosine_similarity")
  ret_task = detectTask(inputText, False)
  for task in ret_task:
    print("Nhiệm vụ số ", cntTask)
    print("Người làm: ", task['people'])
    print("thời gian: ", task['time'])
    print("Nội dung nhiệm vụ: ", task['content'])
    print ("")
    cntTask += 1

  cntTask = 0
  # Trích xuất nhiệm vụ sử dụng keyword
  print("\nTrích xuất nhiệm vụ sử dụng keyword")
  ret_task = detectTaskeyword(inputText, False)
  for task in ret_task:
    print("Nhiệm vụ số ", cntTask)
    print("Người làm: ", task['people'])
    print("thời gian: ", task['time'])
    print("Nội dung nhiệm vụ: ", task['content'])
    print ("")
    cntTask += 1

In [ ]:
text_demo_01 = '''Hôm nay ngày 21 tháng 9 năm 2017 ban chỉ đạo trường THPT Hoài Đức A họp bàn về việc phân công trách nhiệm cho từng thành viên trong Ban chỉ đạo thực hiện công tác phòng chống bệnh sốt xuất huyết.
Ban chỉ đạo gồm có 5 thành viên có tên sau : Nga, Linh, Hùng , Hạnh , Nguyên. Nga hiệu trưởng cũng chính là trưởng ban chỉ đạo phòng chống dịch bệnh thông qua nội dung thực hiện phòng bệnh và giao nhiệm vụ cho từng thành viên. Sau đây là nhiệm vụ cụ thể của từng người: Linh sẽ đảm nhận trách nhiệm hướng dẫn CBGVNV cùng nắm chắc các phương pháp phòng chống bệnh sốt xuất huyết đồng thời chú trọng công tác an toàn thực phẩm,
 kiểm tra vệ sinh, khử khuẩn phòng chống dịch tại trường. Có kế hoạch kiểm tra, giám sát việc thực hiện công tác của từng thành viên trong trường. Deadline 24 tháng 9, 2017.
 Hùng đảm nhận phụ trách công tác y tế, tham mưu, giúp đỡ và cùng phối hợp với nhà trường trong công tác phòng chống bệnh sốt xuất huyết, tổ chức các buổi tuyên truyền về công tác phòng chống bệnh. Deadline 25 tháng 9, 2017.
 Hạnh phối kết hợp cùng các đồng chí trong Ban chỉ đạo theo dõi, giám sát việc thực hiện công tác bệnh sốt xuất huyết. Nguyên đảm nhận tham gia kiểm tra, theo dõi, giám sát việc thực hiện công tác phòng chống bệnh sốt xuất huyết. Thời gian chúng ta thực hiện bắt đầu từ hôm nay. Cuộc họp kết thúc với sự quyết tâm chống dịch cao của các thành viên.'''

In [ ]:
inference(text_demo_01)

Văn bản gốc:  Hôm nay ngày 21 tháng 9 năm 2017 ban chỉ đạo trường THPT Hoài Đức A họp bàn về việc phân công trách nhiệm cho từng thành viên trong Ban chỉ đạo thực hiện công tác phòng chống bệnh sốt xuất huyết. 
Ban chỉ đạo gồm có 5 thành viên có tên sau : Nga, Linh, Hùng , Hạnh , Nguyên. Nga hiệu trưởng cũng chính là trưởng ban chỉ đạo phòng chống dịch bệnh thông qua nội dung thực hiện phòng bệnh và giao nhiệm vụ cho từng thành viên. Sau đây là nhiệm vụ cụ thể của từng người: Linh sẽ đảm nhận trách nhiệm hướng dẫn CBGVNV cùng nắm chắc các phương pháp phòng chống bệnh sốt xuất huyết đồng thời chú trọng công tác an toàn thực phẩm,
 kiểm tra vệ sinh, khử khuẩn phòng chống dịch tại trường. Có kế hoạch kiểm tra, giám sát việc thực hiện công tác của từng thành viên trong trường. Deadline 24 tháng 9, 2017. 
 Hùng đảm nhận phụ trách công tác y tế, tham mưu, giúp đỡ và cùng phối hợp với nhà trường trong công tác phòng chống bệnh sốt xuất huyết, tổ chức các buổi tuyên truyền về công tác phòng ch